# HTML tags position generation

Con la posición base del primer rectángulo, y aquellas posiciones de los saltos necesarios, calcularemos todos los restantes. Las posiciones en los tags de html generados por GIMP vienen dadas por: 'horizontalizq, verticalarriba, horizontalder, verticalabajo'.

Jerarquía de parámetros: switches contienen grupos de bocas infiniband, cada una con 'col' columnas y 'row' filas. 

Input parameters: 
* sw_col: número de 'columnas de switches'
* sw_row: número de 'filas de switches' 
* gr_col: número de 'columnas de grupos'
* gr_row: número de 'filas de grupos'
* col: número de columnas/grupo
* row: número de filas/grupo 

In [1]:
import numpy as np

## Creación de la clase

In [16]:
class mapeador():
    
    def __init__(self, path_input, path_output, dim, sw_col = 1, sw_row = 1, base = 0):
        self.path_input = path_input
        self.path_output = path_output
        self.sw_col = sw_col
        self.sw_row = sw_row
        self.gr_col = dim[0]
        self.gr_row = dim[1]
        self.col = dim[2]
        self.row = dim[3]
        self.base = base
        
        self.importar_map()
        
    def importar_map(self):
        with open(self.path_input, 'r') as file:
            self.imgmap = file.read()
    
    def coordenadas(self):
        #Conseguimos el diccionario con las cordenadas del rectángulo inicial y los saltos necesarios
        coords_dict = {}
        split = [split.split('\" alt=\"') for split in self.imgmap.split('coords=\"')[1:]]
        
        for coords in split:
            alt = coords[1].split('\"  nohref')[0] #Careful: dos espacios entre \" y nohref
            coords_dict[int(alt)] = coords[0]
        
        return coords_dict
        
    def distancias(self):
        #Calculamos las distancias al rectángulo inicial (base)
        coords_dict = self.coordenadas()
        
        dist_dict = {}
        pos_base = [int(pos) for pos in coords_dict[1].split(",")]
        
        for entry in coords_dict.keys():
            if entry == 1:
                continue
            pos = [int(pos) for pos in coords_dict[entry].split(",")]
            dist_dict[entry] = np.subtract(pos,pos_base)
        
        #dist_good = {entry: list(dist_dict[entry][:2]) + list(dist_dict[entry][:2]) for entry in dist_dict.keys()} # Nos quedamos con las dos primeras entradas, y las repetimos.
        
        return dist_dict, pos_base
        
    def todas_posiciones(self):
        #obtenemos un diccionario alt <---> posiciones de los rectángulos
        dist_good, pos_base = self.distancias()
    
        alt = 1
        all_pos = {}
        all_pos[1 + self.base] = np.array(pos_base)

        def calcular_posicion(alt,i,j,k,l,m,n):
            #Calculo del siguiente rectángulo.
            change = np.multiply(i,dist_good[2]) + np.multiply(j,dist_good[3]) + np.multiply(k,dist_good[self.col * self.row + 1])
            if l: 
                change += np.multiply(l,dist_good[self.col * self.row * self.gr_col + 1])
            if m:
                change += np.multiply(m,dist_good[self.col * self.row * self.gr_col * self.gr_row + 1])
            if n: 
                change += np.multiply(n,dist_good[self.col * self.row * self.gr_col * self.gr_row * self.sw_col + 1])


            pos = np.add(pos_base, change)
            alt += 1
            return alt, pos 

        for n in range(self.sw_row):
            for m in range(self.sw_col):
                for l in range(self.gr_row):
                    for k in range(self.gr_col):
                        for j in range(self.col):
                            for i in range(self.row):
                                if i == 0 and j == 0 and k == 0 and l == 0 and m == 0 and n == 0:
                                    continue
                                alt, pos = calcular_posicion(alt,i,j,k,l,m,n)
                                all_pos[alt + self.base] = pos
        print(all_pos)
        
        return all_pos
    
    def preparar_output(self):
        #Eliminar los tags que ahora sustituiremos
        if self.base != 0:
            with open(self.path_output, 'r') as file:
                self.imgmap = file.read()
        
        splitmap = self.imgmap.split('\n')
        finalmap = [] 
        if self.base == 0: #borrar lineas en el fichero generado a mano
            for split in splitmap:
                if split.find('<area'):
                    finalmap.append(split)
        else: #incorporar a un fichero más filas a partir de un numero base
            finalmap = splitmap
            
        return finalmap
    
    def generar_y_guardar(self):
        # Insertar los tags en la lista que irá al fichero
        all_pos = self.todas_posiciones()
        finalmap = self.preparar_output()
        
        for entry in all_pos.keys():
            intlist = all_pos[entry].tolist()
            strlist = [str(num) for num in intlist]
            val = ",".join(strlist)
            tag = f'<area shape="rect" coords="{val}" alt="{entry}"  nohref="nohref" />'
            finalmap.insert(-2,tag)
        
        with open(self.path_output, 'w') as file:
            file.write('\n'.join(finalmap))
        
        print("\nalright!")

### Caso 1: switch leafs del switch SX6536

In [17]:
#Detalles switch:
gr_col_1 = 3
gr_row_1 = 1
col_1 = 3
row_1 = 2

In [18]:
#Input/output:
path_input_1 = 'mapeos/[SX6536_detalle] (imported).map'
path_output_1 = 'mapeos/SX6536_detalle.map'

In [19]:
leafSX6536 = mapeador(path_input_1, path_output_1, [gr_col_1, gr_row_1, col_1, row_1])

In [20]:
leafSX6536.generar_y_guardar()

{1: array([108,  61, 141,  76]), 2: array([108,  84, 142,  99]), 3: array([143,  61, 176,  76]), 4: array([143,  84, 177,  99]), 5: array([178,  61, 211,  76]), 6: array([178,  84, 212,  99]), 7: array([219,  61, 251,  76]), 8: array([219,  84, 252,  99]), 9: array([254,  61, 286,  76]), 10: array([254,  84, 287,  99]), 11: array([289,  61, 321,  76]), 12: array([289,  84, 322,  99]), 13: array([330,  61, 361,  76]), 14: array([330,  84, 362,  99]), 15: array([365,  61, 396,  76]), 16: array([365,  84, 397,  99]), 17: array([400,  61, 431,  76]), 18: array([400,  84, 432,  99])}

alright!


### Caso 2: switch SX6036

In [21]:
#Detalles switch:
gr_col_2 = 3
gr_row_2 = 1
col_2 = 6
row_2 = 2

In [22]:
#Input/output:
path_input_2 = 'mapeos/[StorageReview-Mellanox-SX6036-InfiniBand-Switch-Front] (imported).map'
path_output_2 = 'mapeos/SX6036_IB_Front.map'

In [23]:
switchSX6036 = mapeador(path_input_2, path_output_2, [gr_col_2, gr_row_2, col_2, row_2])
switchSX6036.generar_y_guardar()

{1: array([ 70,  51, 105,  67]), 2: array([ 71,  76, 106,  92]), 3: array([108,  51, 142,  67]), 4: array([109,  76, 143,  92]), 5: array([146,  51, 179,  67]), 6: array([147,  76, 180,  92]), 7: array([184,  51, 216,  67]), 8: array([185,  76, 217,  92]), 9: array([222,  51, 253,  67]), 10: array([223,  76, 254,  92]), 11: array([260,  51, 290,  67]), 12: array([261,  76, 291,  92]), 13: array([300,  53, 334,  67]), 14: array([301,  78, 335,  92]), 15: array([338,  53, 371,  67]), 16: array([339,  78, 372,  92]), 17: array([376,  53, 408,  67]), 18: array([377,  78, 409,  92]), 19: array([414,  53, 445,  67]), 20: array([415,  78, 446,  92]), 21: array([452,  53, 482,  67]), 22: array([453,  78, 483,  92]), 23: array([490,  53, 519,  67]), 24: array([491,  78, 520,  92]), 25: array([530,  55, 563,  67]), 26: array([531,  80, 564,  92]), 27: array([568,  55, 600,  67]), 28: array([569,  80, 601,  92]), 29: array([606,  55, 637,  67]), 30: array([607,  80, 638,  92]), 31: array([644,  5

### Caso 3: switch SX6536 (parte 1)

Vamos a dividir el switch SX6536 en dos partes, ya que hay una separación con una barra de metal entre una parte con 12 switches encima de una parte con 24 (en ambos casos, en dos columnas). Esta irregularidad en la cantidad de switches existente en ambas partes hace dificil automatizarlo. Para no comerme la cabeza, simplemente he añadido un parámetro 'base' a partir del cual empezar a contar, en este caso el segundo grupo de switches comienza en el 217 (ese sería el 1, por lo que la base es 216).

In [24]:
#Detalles switch:
sw_col_3 = 2
sw_row_3 = 1
gr_col_3 = 3
gr_row_3 = 6
col_3 = 3
row_3 = 2

In [25]:
#Input/output:
path_input_3 = 'mapeos/[sx6536] (imported).map'
path_output_3 = 'mapeos/sx6536_full.map'

In [26]:
switchSX6036 = mapeador(path_input_3, path_output_3, [gr_col_3, gr_row_3, col_3, row_3], sw_col = sw_col_3, sw_row = sw_row_3)
switchSX6036.generar_y_guardar()

{1: array([ 37, 187,  47, 193]), 2: array([ 37, 197,  47, 203]), 3: array([ 51, 187,  61, 193]), 4: array([ 51, 197,  61, 203]), 5: array([ 65, 187,  75, 193]), 6: array([ 65, 197,  75, 203]), 7: array([ 83, 187,  93, 193]), 8: array([ 83, 197,  93, 203]), 9: array([ 97, 187, 107, 193]), 10: array([ 97, 197, 107, 203]), 11: array([111, 187, 121, 193]), 12: array([111, 197, 121, 203]), 13: array([129, 187, 139, 193]), 14: array([129, 197, 139, 203]), 15: array([143, 187, 153, 193]), 16: array([143, 197, 153, 203]), 17: array([157, 187, 167, 193]), 18: array([157, 197, 167, 203]), 19: array([ 37, 223,  47, 229]), 20: array([ 37, 233,  47, 239]), 21: array([ 51, 223,  61, 229]), 22: array([ 51, 233,  61, 239]), 23: array([ 65, 223,  75, 229]), 24: array([ 65, 233,  75, 239]), 25: array([ 83, 223,  93, 229]), 26: array([ 83, 233,  93, 239]), 27: array([ 97, 223, 107, 229]), 28: array([ 97, 233, 107, 239]), 29: array([111, 223, 121, 229]), 30: array([111, 233, 121, 239]), 31: array([129, 22

### Caso 3: switch SX6536 (parte 2)

In [27]:
#Detalles switch:
sw_col_32 = 2
sw_row_32 = 1
gr_col_32 = 3
gr_row_32 = 12
col_32 = 3
row_32 = 2
base = 216

In [28]:
#Input/output:
path_input_32 = 'mapeos/[sx6536] (imported) - 2.map'
path_output_32 = 'mapeos/sx6536_full.map'

In [29]:
switchSX6036_2 = mapeador(path_input_32, path_output_32, [gr_col_32, gr_row_32, col_32, row_32], sw_col = sw_col_32, sw_row = sw_row_32, base = base)
switchSX6036_2.generar_y_guardar()

{217: array([ 37, 405,  47, 411]), 218: array([ 37, 415,  47, 421]), 219: array([ 51, 405,  61, 411]), 220: array([ 51, 415,  61, 421]), 221: array([ 65, 405,  75, 411]), 222: array([ 65, 415,  75, 421]), 223: array([ 83, 405,  93, 411]), 224: array([ 83, 415,  93, 421]), 225: array([ 97, 405, 107, 411]), 226: array([ 97, 415, 107, 421]), 227: array([111, 405, 121, 411]), 228: array([111, 415, 121, 421]), 229: array([129, 405, 139, 411]), 230: array([129, 415, 139, 421]), 231: array([143, 405, 153, 411]), 232: array([143, 415, 153, 421]), 233: array([157, 405, 167, 411]), 234: array([157, 415, 167, 421]), 235: array([ 37, 441,  47, 447]), 236: array([ 37, 451,  47, 457]), 237: array([ 51, 441,  61, 447]), 238: array([ 51, 451,  61, 457]), 239: array([ 65, 441,  75, 447]), 240: array([ 65, 451,  75, 457]), 241: array([ 83, 441,  93, 447]), 242: array([ 83, 451,  93, 457]), 243: array([ 97, 441, 107, 447]), 244: array([ 97, 451, 107, 457]), 245: array([111, 441, 121, 447]), 246: array([1